<a href="https://colab.research.google.com/github/sergey031/DZ_MLP/blob/master/04_hw_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Кластеризация. Практическая работа 2

## Совсем простая рекомендательная система

На основе оценок аниме, которые ставят пользователи систем типа [MAL](https://myanimelist.net/), можно строить разные кластеры данных:
- кластеры похожих людей. Похожие значит, что эти люди ставят похожие рейтинги аниме.
- кластеры похожих аниме. Похожие значит что люди оценивают их похоже.
- кластеры похожих жанров. Но похожие не в обычном смысле, а в смысле, что люди которые смотрят жанр А любят смотреть жанр Б.

и т.д.

### Полезная литература

- [Лекция 8. Рекомендательный системы](https://www.youtube.com/watch?v=Te_6TqEhyTI&t=4s).
- [Туториал по рекомендательным системам](http://nbviewer.jupyter.org/urls/gitlab.7bits.it/isiganov/ml-course/raw/master/week05/theory/05-01-clustering.ipynb?inline=false)
- [ODS: Обучение без учителя: PCA и кластеризация](https://habrahabr.ru/company/ods/blog/325654/)
- [Интересные алгоритмы кластеризации, часть первая: Affinity propagation](https://habrahabr.ru/post/321216/) и другие статьи цикла
- [Глава 7: кластеризация и визуализация. К. В. Воронцов](http://www.machinelearning.ru/wiki/images/6/6d/Voron-ML-1.pdf)
- [Документация sklearn.clustering](http://scikit-learn.org/stable/modules/clustering.html)
- [K-Means Clustering - The Math of Intelligence. Siraj Raval](https://www.youtube.com/watch?v=9991JlKnFmk) объяснение с программированием KMeans вручную



In [1]:
from sklearn import datasets
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.cluster import KMeans

from tqdm import tqdm
# from tqdm import tqdm_notebook as tqdm # Раскоментируйте если прогресс бар будет странно работать

%matplotlib inline

# Зафиксируем случайность, чтобы у нас получались одинаковые результаты.
np.random.seed(seed=42)

## Анализ отзывов аниме

Возьмем датасет с рейтингами аниме: https://www.kaggle.com/CooperUnion/anime-recommendations-database  
Кстати, вы можете посмотреть kernels - это jupyter notebooks, в которых другие люди тоже делали что-то с этим датасетом.

```
Anime.csv

anime_id - myanimelist.net's unique id identifying an anime.
name - full name of anime.
genre - comma separated list of genres for this anime.
type - movie, TV, OVA, etc.
episodes - how many episodes in this show. (1 if movie).
rating - average rating out of 10 for this anime.
members - number of community members that are in this anime's "group".


Rating.csv

user_id - non identifiable randomly generated user id.
anime_id - the anime that this user has rated.
rating - rating out of 10 this user has assigned (-1 if the user watched it but didn't assign a rating).
```

In [3]:
colab = False  # True если используте google colab
if colab:
    from google.colab import drive
    drive.mount('/content/drive/')

In [5]:
if colab:
    anime = pd.read_csv('anime.csv')
else:
    anime = pd.read_csv('anime.csv')
anime.dropna(inplace=True)
print(anime.shape)
anime.head()

(12017, 7)


,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [7]:
if colab:
    ratings = pd.read_csv('rating.csv')
else:
    ratings = pd.read_csv('rating.csv')
ratings

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1
...,...,...,...
1176141,11234,3002,8
1176142,11234,3036,7
1176143,11234,4182,7
1176144,11234,4186,8


Датасет очень большой и грязный. Некоторые действия с этим датасетом будут требовать много оперативной памяти(>6 Гб).

## Подготовка данных

Во первых, в датасете есть много -1. Оценки -1 и 0 на MAL нет.

Здесь -1 означает что человек посмотрел это аниме, но не выставил оценку.

Такие записи из `ratings` стоит выбросить, так как в нашем случае они особо не помогут.

Но и не помешают серьезно. Если хотите оставьте их, только нужно заменить все -1 на 0, так как дальше нам понадобится посчитать среднее, а -1 или 0, в отличие от `np.nan`, повлияют на среднее.

### 1. Избавьтесь от -1

In [8]:
import pandas as pd

colab = True  # True если используте google colab
if colab:
    ratings = pd.read_csv('rating.csv')
else:
    ratings = pd.read_csv('rating.csv')


ratings = ratings[ratings['rating'] != -1]

## Критерий Шавене (Chauvenet)

[Теория](https://www.youtube.com/watch?v=Fy9pHH3ykPE&list=PLLyuiBK_HOLPfRVN6r9305FKXq1ravbbX)

$$ erfc(\frac{|P_i - mean(P)|}{S_p})  < \frac{1}{2n}$$

$ S_p - отклонение $

Готовой реализации в библиотеках нет, поэтому придется написать самим(но если найдете можете использовать).

### 2. Напишите функцию, которая принимает на вход массив, считает критерий Шавене и возвращает булеву маску.

Функция `erfc` есть в sklearn.

In [10]:
import numpy as np
from scipy.stats import shapiro

def shapiro_wilk_test(data, alpha=0.05):
    statistic, p_value = shapiro(data)
    mask = p_value > alpha
    return mask

data = np.random.normal(0, 1, 100)
is_normal = shapiro_wilk_test(data)
print(is_normal)


True


Для начала давайте посмотрим на таблицу рейтингов.

### 3. Сделайте новую таблицу `count_reviews` где индексами будет `user_id` а значением будет количество просмотренных им аниме.


**Hint** Используйте [groupby](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html) и [count](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.count.html).

In [11]:
count_reviews = ratings.groupby('user_id')['anime_id'].count().reset_index()
count_reviews.columns = ['user_id', 'count_reviews']

### 4. Используйте функцию chauvenet и найдите все выбросы.

**Hint:** Так как chauvenet возвращает маску используйте оператор `[]` (подробнее смотрите в 1 теории по pandas и numpy).

**Hint:** Используйте [values](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.values.html).

In [12]:
count_reviews_array = count_reviews['count_reviews'].values
mask = chauvenet(count_reviews_array)

outlier_users = count_reviews['user_id'][mask].values

### 5. Ответьте на вопросы

#### Кого критерий посчитал выбросом?

#### Почему критерий посчитал их выбросом?

#### Нужна ли им вообще рекомендательная система?


**Ответы:**

In [45]:
Критерий Шавене определил в качестве выбросов пользователей с очень большим количеством просмотренных аниме (более 500).
Такое большое количество просмотров является статистически маловероятным и нехарактерным для обычных пользователей, поэтому критерий отнес этих пользователей к выбросам.
Скорее всего таким пользователям не нужна рекомендательная система, так как они уже посмотрели огромное количество аниме и вряд ли будут ориентироваться на рекомендации.

Если все было правильно `bad_user_threshold` больше 500.

Нужно выбросить всех людей у которых число просмотренных аниме больше или равно `bad_user_threshold`.

### 6. Переименнуйте столбец из таблицы `count_reviews` в `count_reviews` (он там единственный). Соедините `count_reviews` и `ratings` по столбцу `user_id`. И оставьте в `ratings` только тех кто посмотрел меньше `bad_user_threshold`  

In [45]:
import pandas as pd

count_reviews = count_reviews.rename(columns={'count_reviews': 'count_reviews'})
merged_data = pd.merge(ratings, count_reviews, on='user_id')
bad_user_threshold = 500
filtered_data = merged_data[merged_data['count_reviews'] < bad_user_threshold]
filtered_data.drop('count_reviews', axis=1, inplace=True)
print(filtered_data)

Осталось все равно слишком много пользователей.

### 7. Удалите из таблицы всех юзеров, у которых количество просмотров меньше медианного значения. То есть мы удалим половину юзеров.

Так как они посмотрели слишком мало, чтобы мы на них могли основывать свои советы.

In [18]:

ratings

,user_id,anime_id,rating
47,1,8074.0,10.0
81,1,11617.0,10.0
83,1,11757.0,10.0
101,1,15451.0,10.0
153,2,11771.0,10.0
...,...,...,...
1536211,14837,31964.0,8.0
1536212,14837,32189.0,9.0
1536213,14837,32282.0,9.0
1536214,14837,33028.0,9.0


In [44]:
import pandas as pd

median_count_reviews = filtered_data['count_reviews'].median()
filtered_data = filtered_data[filtered_data['count_reviews'] >= median_count_reviews]
print(filtered_data)

Теперь рассмотрим таблицу `anime`.


Так же применим критерий шавене.

Искать выбросы стоит по столбцу `rating` или по `members` или по обоим сразу.

### 8. Используйте функцию chauvenet и найдите все выбросы среди аниме. И удалите их.

**Hint** Используйте [drop](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html) и [index](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.index.html)

In [20]:
import pandas as pd
import numpy as np
from scipy.stats import shapiro

def chauvenet(data, alpha=0.05):
    mean = np.mean(data)
    std_dev = np.std(data, ddof=1)
    n = len(data)
    p_values = 2 * (1 - shapiro_wilk_test(data))
    criterion = n * p_values / (2 * np.sqrt(2 * np.log(n)))
    outliers = abs(data - mean) > criterion

    return outliers
rating_outliers = chauvenet(anime['rating'])
members_outliers = chauvenet(anime['members'])
all_outliers = rating_outliers | members_outliers
anime = anime[~all_outliers]
print(anime)


       anime_id                                               name  \
41        32366                         Gintama°: Aizome Kaori-hen   
212       31490                               One Piece Film: Gold   
227        6467  Detective Conan Movie 14: The Lost Ship in the...   
229       22507                              Initial D Final Stage   
231       19123  One Piece: Episode of Merry - Mou Hitori no Na...   
...         ...                                                ...   
11239      5719                                        Sex Pistols   
11251      5784                                Ai no Kusabi (2012)   
11285      4473                                        Shoujo Sect   
11310      8918                                Princess Lover! OVA   
11681       651                   Green Green Thirteen: Erolutions   

                                                   genre     type episodes  \
41                                        Comedy, Parody      OVA        2   
212

/usr/local/lib/python3.10/dist-packages/scipy/stats/_morestats.py:1882: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


### 9. Ответье на вопросы

#### Что критерий посчитал выбросом?

#### Почему критерий посчитал их выбросом?

#### Можем ли мы как то использовать эти аниме в нашей рекомендательное системе?

**Ответы:**

## Кластеризация по жанрам

Данные о жанре хранятся как строка, разделенная запятой `,` . Но нам нужны сами жанры. Придется поколдовать и разделить эту строку на элементы.

In [21]:
from itertools import chain
'''
Нам нужна функция flatmap.
Flatmap получает на вход список, на каждом элементе вызывает функцию f, которая возвращает другой список.
В результате получается список списков. В конфе происходит flatten - уплощение списка скписков в один список.
'''
def flatmap(f, items):
    return chain.from_iterable(map(f, items))

# пример использования
list(flatmap(lambda x: [0, x , x*x], [1,2,3,4,5]))
# Первый шаг: [[0, 1, 1], [0, 2, 4], [0, 3, 9], [0, 4, 16], [0, 5, 25]]

[0, 1, 1, 0, 2, 4, 0, 3, 9, 0, 4, 16, 0, 5, 25]

In [22]:
# создаем функцию, которая просто разбивает строку по символу ", " на подстроки
def genre_splitter(genre_names):
    return genre_names.split(", ")

m_uniq = anime['genre'].unique() # смотрим сколько всего уникальных комбинация genres есть в датасете
print("m_uniq[0:10] = {}\nlen= {}\n".format(m_uniq[0:10], len(m_uniq))) # как видим комбинаций очень много, так как там все композиции

genres = set(flatmap(genre_splitter, m_uniq)) # разбиваем все genres на составные части и генерируем один массив из всех жанров. Строим по массиву множество уникальных жанров

genres = list(genres) # множество превращаем в список
print("Genres={}\nlen={}".format(genres, len(genres)))

m_uniq[0:10] = ['Comedy, Parody' 'Action, Adventure, Comedy, Drama, Fantasy, Shounen'
 'Action, Mystery, Police, Shounen' 'Cars, Seinen, Sports'
 'Action, Adventure, Comedy, Drama, Fantasy, Shounen, Super Power'
 'Drama, Shoujo' 'Comedy, Drama, Shounen, Sports'
 'Comedy, School, Shounen, Sports'
 'Fantasy, Sci-Fi, Shounen, Slice of Life'
 'Adventure, Comedy, Mystery, Police, Shounen']
len= 299

Genres=['Mystery', 'Ecchi', 'Drama', 'Horror', 'Military', 'Hentai', 'Romance', 'Demons', 'Slice of Life', 'Action', 'Martial Arts', 'Dementia', 'Yuri', 'Fantasy', 'Shoujo Ai', 'Historical', 'Supernatural', 'Super Power', 'Cars', 'Shounen', 'School', 'Shounen Ai', 'Mecha', 'Parody', 'Samurai', 'Sports', 'Space', 'Music', 'Seinen', 'Sci-Fi', 'Psychological', 'Thriller', 'Game', 'Shoujo', 'Magic', 'Police', 'Kids', 'Comedy', 'Harem', 'Adventure', 'Josei', 'Yaoi', 'Vampire']
len=43


### 10. Создадим новую таблицу, где в колонках будет жанр, в строках аниме, а в ячейках 1 если у фильма есть этот жанр и 0 в противном случае.

Такой формат таблиц называтеся one-hot-encoding. Только в нашем случае в каждой строке будет не одна единица, а несколько, так как у аниме как правило несолько жанров.

Уточнение: жанры должны быть индексами столбцов (columns), а id аниме - индексами строк (index)

In [23]:
import pandas as pd

anime['genre'] = anime['genre'].str.split(', ')
one_hot_encoding = pd.DataFrame(0, index=anime.index, columns=[])
for index, row in anime.iterrows():
    for genre in row['genre']:
        one_hot_encoding.at[index, genre] = 1
print(one_hot_encoding)


       Comedy  Parody  Action  Adventure  Drama  Fantasy  Shounen  Mystery  \
41        1.0     1.0     NaN        NaN    NaN      NaN      NaN      NaN   
212       1.0     NaN     1.0        1.0    1.0      1.0      1.0      NaN   
227       NaN     NaN     1.0        NaN    NaN      NaN      1.0      1.0   
229       NaN     NaN     NaN        NaN    NaN      NaN      NaN      NaN   
231       1.0     NaN     1.0        1.0    1.0      1.0      1.0      NaN   
...       ...     ...     ...        ...    ...      ...      ...      ...   
11239     1.0     NaN     NaN        NaN    1.0      NaN      NaN      NaN   
11251     NaN     NaN     NaN        NaN    1.0      NaN      NaN      NaN   
11285     1.0     NaN     NaN        NaN    NaN      NaN      NaN      NaN   
11310     NaN     NaN     NaN        NaN    NaN      NaN      NaN      NaN   
11681     1.0     NaN     NaN        NaN    NaN      NaN      NaN      NaN   

       Police  Cars  ...  Samurai  Psychological  Thriller  Jos

Итак, у нас есть следующие таблицы:
- Жанры аниме - в строчках аниме, в столбцах жанр аниме, а в ячейках 0 или 1.
- Рейтинги - в строчках пользователи, в столбцах id аниме и рейтинг

Нужно получить другую таблицу, в которой в строках будут пользователи, а в столбцах жанры аниме. А в ячейках средняя оценка жанра этим пользователем.

Средняя оценка жанра вычисляется следующим образом: берём все отзывы пользователя. Группируем все его отзывы по жанрам и считаем средний рейтинг, который он ставит аниме с данным жанром.

Выполним следущие шаги.

### 11. Соединим две таблицы:<br>
1. жанры по каждому аниме<br>
2. оценки аниме от людей. Кстати, один человек мог посмотреть 1 аниме или 100, но не все!<br>

Получим таблицу, где строк будет N*M штук, где N - количество юзеров и M - количество аниме

In [43]:
import pandas as pd
import numpy as np

user_anime_ratings = pd.pivot_table(ratings, index='user_id', columns='anime_id', values='rating')
user_anime_ratings = user_anime_ratings.fillna(0)
genre_ratings = user_anime_ratings.dot(one_hot_encoding)
print(genre_ratings)

С такой таблицей `(N*M) * G` вы всё еще не можем работать.  
### 12. Сгруппируем строки по пользователям (колонка `userId`).  В группах посчитаем среднюю оценку на жанр. А если пользователь не смотрел фильм, то поставим ему `-1` в соответсвующую ячейку.
Чтобы посчитать среднее(mean) без учета непросмотренных аниме замените все `0` на `np.NaN`

In [42]:
import pandas as pd
import numpy as np

user_anime_ratings = pd.pivot_table(ratings, index='user_id', columns='anime_id', values='rating')
user_anime_ratings = user_anime_ratings.replace(0, np.nan)
user_genre_ratings = user_anime_ratings.dot(one_hot_encoding)
user_genre_ratings = user_genre_ratings.fillna(-1)
print(user_genre_ratings)

Так как некоторые пользователи не смотрели ничего из некоторых жанров, в данных осталось много `np.NaN`
### 13. Заполните все NaN на -1

**Hint** [fillna](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html)

In [41]:
user_genre_ratings = user_genre_ratings.fillna(-1)



Прежде чем начать обучать kMeans...

### 14. Отмасштабируйте признаки.

Как мы знаем по лекции, метрическим алгоритмам, одним из которых и является kMeans, лучше подавать на вход данные одного масштаба.  Этим и занимается метод MinMaxScaler из sklearn.

[Документация](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html)

Алгоритм его работы:
```
X_std = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
X_scaled = X_std * (max - min) + min
```

In [41]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_user_genre_ratings = scaler.fit_transform(user_genre_ratings)
scaled_user_genre_ratings = pd.DataFrame(scaled_user_genre_ratings, columns=user_genre_ratings.columns, index=user_genre_ratings.index)
print(scaled_user_genre_ratings)

### 15.Натренируйте kMeans с 10 кластерами на полученных данных

In [40]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=10)
kmeans.fit(scaled_user_genre_ratings)
cluster_labels = kmeans.labels_
print(cluster_labels)

### 16. Нарисуйте на графике центры кластеров нашего датасета оценок фильмов.

В нем будет 10 строчек - 10 кластеров. И 43 столбцов - 43 жанров фильмов.

Из графика мы поймем какие жанры обычно смотрят вместе. По сути мы получили кластеры предпочтений людей.

**Hint** [Heatmap](https://seaborn.pydata.org/generated/seaborn.heatmap.html)

In [39]:
import seaborn as sns
import matplotlib.pyplot as plt

cluster_centers = kmeans.cluster_centers
cluster_center_df = pd.DataFrame(cluster_centers, columns=scaled_user_genre_ratings.columns)
plt.figure(figsize=(12, 8))
sns.heatmap(cluster_center_df, cmap='YlGnBu')
plt.title("Центры кластеров по жанрам")
plt.show()

# Как выбрать нужное число кластеров

Такие методы как KMeans, Spectral clustering, Ward hierarchical clustering, Agglomerative clustering требуют количество кластеров как параметр. Это так называемый гипер-параметр, и его должен подбирать человек. Но на что человеку опираться при выборе? На некоторый функционал "качества"!

Вспомним идею кластеризации:
- минимизация внутрикластерного расстояния
- максимизация межкластерного расстояния

Другими словами - кучки кучнее и дальше друг от друга.

Логично, что мы хотим, чтобы точки распологались кучно возле центров своих кластеров. Но вот незадача: минимум такого функционала будет достигаться тогда, когда кластеров столько же, сколько и точек (то есть каждая точка – это кластер из одного элемента). Для решения этого вопроса (выбора числа кластеров) часто пользуются такой эвристикой: выбирают то число кластеров, начиная с которого описанный функционал $ J(C) $ падает "уже не так быстро". Или более формально: $$ D(k) = \frac{|J(C_k) - J(C_{k+1})|}{|J(C_{k-1}) - J(C_k)|}  \rightarrow \min\limits_k $$

Где, в случае kMeans $$ J(C) = \sum_{k=1}^K\sum_{i~\in~C_k} ||x_i - \mu_k|| \rightarrow \min\limits_C,$$ - сумма квадратов расстояний от точек до центроидов кластеров, к которым они относятся

#### Эта ячейка может выполнятся долго!

In [38]:
inertia = []
N = 30
for k in tqdm(range(1, N)):
    kmeans = KMeans(n_clusters=k).fit(scaler.fit_transform(df))
    inertia.append(np.sqrt(kmeans.inertia_))
plt.figure(figsize=(10,7))
plt.plot(range(1, N), inertia, marker='s');
plt.xlabel('$k$')
plt.ylabel('$J(C_k)$')

## Коэффициент силуэта

Данный коэффициент не предполагает знания истинных меток объектов, и позволяет оценить качество кластеризации, используя только саму (неразмеченную) выборку и результат кластеризации.

Сначала силуэт определяется отдельно для каждого объекта. Обозначим через $a$ - среднее расстояние от данного объекта до объектов из того же кластера, через $b$ - среднее расстояние от данного объекта до объектов из ближайшего кластера (отличного от того, в котором лежит сам объект). Тогда силуэтом данного объекта называется величина: $$s = \frac{b - a}{\max(a, b)}.$$ Силуэтом выборки называется средняя величина силуэта объектов данной выборки. Таким образом, силуэт показывает, насколько среднее расстояние до объектов своего кластера отличается от среднего расстояния до объектов других кластеров. Данная величина лежит в диапазоне $[-1, 1]$. Значения, близкие к -1, соответствуют плохим (разрозненным) кластеризациям, значения, близкие к нулю, говорят о том, что кластеры пересекаются и накладываются друг на друга, значения, близкие к 1, соответствуют "плотным" четко выделенным кластерам. Таким образом, чем больше силуэт, тем более четко выделены кластеры, и они представляют собой компактные, плотно сгруппированные облака точек.

С помощью силуэта можно выбирать оптимальное число кластеров $k$ (если оно заранее неизвестно) - выбирается число кластеров, максимизирующее значение силуэта. В отличие от предыдущих метрик, силуэт зависит от формы кластеров, и достигает больших значений на более выпуклых кластерах, получаемых с помощью алгоритмов, основанных на восстановлении плотности распределения.

#### Эта ячейка может выполняться долго!

In [31]:
from sklearn.metrics import silhouette_score, silhouette_samples

def draw_sil_score(X, range_n_clusters=[2, 3, 4, 5, 6, 10, 12, 13, 20]):
    scores = []
    for n_clusters in tqdm(range_n_clusters):
        clusterer = KMeans(n_clusters=n_clusters, random_state=10)
        cluster_labels = clusterer.fit_predict(X)
        silhouette_avg = silhouette_score(X, cluster_labels)
        scores.append(silhouette_avg)
    plt.plot(range_n_clusters, scores)
    return range_n_clusters[np.argmax(scores)]

In [ ]:
draw_sil_score(scaler.fit_transform(df), range(2, 30))

### 17. Выберите количество кластеров `k` по методам выше. Натренируйте kMeans и снова нарисуйте heatmap.

In [37]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

inertia = []


for k in range(1, 21):
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(scaled_user_genre_ratings)
    inertia.append(kmeans.inertia_)

plt.figure(figsize=(8, 6))
plt.plot(range(1, 21), inertia, marker='o', linestyle='--')
plt.xlabel('Количество кластеров (k)')
plt.ylabel('Инерция')
plt.title('Метод локтя для выбора оптимального k')
plt.grid(True)
plt.show()

### 18. Порекомендуйте что-нибудь абстрактному пользователю.
Это можно сделать разными способами. Как это сделать подумайте сами.

Если затрудняетесь реализовать это в коде, распишите словами как вы бы это сделали.

Возможные варианты решения:
 * в каждом кластере отсортировать жанры по тому, насколько жанр важен.
 * взять каждый кластер -> получить все аниме, которые смотрят в этом кластере -> отсортировать по рейтину.



In [36]:
abstract_user_cluster = 3
users_in_cluster = user_genre_ratings[cluster_labels == abstract_user_cluster].index
anime_watched_by_cluster = user_anime_ratings.loc[users_in_cluster]
average_ratings_in_cluster = anime_watched_by_cluster.mean()
recommended_anime = average_ratings_in_cluster.sort_values(ascending=False)
top_N_recommendations = recommended_anime.head(10)
print(top_N_recommendations)

### Extra. Попробуйте как-нибудь улучшить эту рекомендашку. Приведите код или рассуждения на эту тему.

Если писать код, то можно:
 * каждому жанру присвоить свой вес, так как одних жанров сильно много и у них разная смысловая нагрузка. Комедии и экшн встречаются очень часто и врядли кто-то только из-за этих жанров будет смотреть аниме.
 * предсказывать не по жанрам, а по аниме. Там получится очень большая размерность, так как нужно сделать one-hot-encoding по аниме, но может это даст лучше результат(спойлер: нет). (И для этого надо сделать 4 join'а, что, возможно, убьет ваш компьютер или/и мозг)

In [35]:
abstract_user_cluster = 3
users_in_cluster = user_genre_ratings[cluster_labels == abstract_user_cluster].index
anime_watched_by_cluster = user_anime_ratings.loc[users_in_cluster]
average_ratings_in_cluster = anime_watched_by_cluster.mean()
weighted_ratings = average_ratings_in_cluster * genre_weights
recommended_anime = weighted_ratings.sort_values(ascending=False)
top_N_recommendations = recommended_anime.head(10)
print(top_N_recommendations)